In [47]:
import pandas as pd
import numpy as np
from IPython.display import display

# Load your data
csv_path = r"C:\STUDIE\4. Semester\Dataprojekt\Dataprojekt - Solo\Overbite_Overjet\Output\Pixel Matrix\NyeBilleder\55K\Pixel_matrix_with_distances.csv"
xlsx_path = r"C:\STUDIE\4. Semester\Dataprojekt\Dataprojekt - Solo\Data\Raw Data\2024-04-08 Test data for overbite classification.xlsx"

df_csv = pd.read_csv(csv_path)
df_excel = pd.read_excel(xlsx_path)

# Clean filenames for merge
df_excel["Filename"] = df_excel["Filename"].str.replace(".png", "", regex=False)

# Only use filenames that exist in both files
df_excel_matched = df_excel[df_excel["Filename"].isin(df_csv["Filename"])]

# Merge metadata and predictions
df_merged = pd.merge(df_excel_matched, df_csv, on="Filename", how="left")

# Extract jaw and side
df_merged["Jaw"] = df_merged["Filename"].str.extract(r"_(upper|lower)")
df_merged["Side"] = df_merged["Filename"].str.extract(r"_(left|right)")
df_merged["Base_ID"] = df_merged["Filename"].str.extract(r"([A-Z0-9]+)")

# Flip Y and compute Y* for upper jaw
df_merged["Y_flipped"] = 1023 - df_merged["Y_Refined"]
df_merged["Y_star"] = df_merged["Y_flipped"] - df_merged["Y_vertical_translate"]

# Separate upper and lower jaw data
df_upper = df_merged[df_merged["Jaw"] == "upper"].copy()
df_lower = df_merged[df_merged["Jaw"] == "lower"].copy()

# Merge by Base_ID and Side only (left-left, right-right)
df_pair = pd.merge(
    df_upper[["Base_ID", "Side", "Y_star", "Class"]],
    df_lower[["Base_ID", "Side", "Y_Refined"]],
    on=["Base_ID", "Side"],
    suffixes=("_upper", "_lower")
)

# Calculate overbite in mm
df_pair["overbite_mm"] = (df_pair["Y_star"] - df_pair["Y_Refined"]) * 0.08

# Classify overbite
def classify_overbite(mm):
    if mm < 1:
        return "A"
    elif mm < 2:
        return "B"
    elif mm < 3:
        return "C"
    elif mm < 4:
        return "D"
    else:
        return "E"

df_pair["Predicted_Class"] = df_pair["overbite_mm"].apply(classify_overbite)

# Compare to ground truth
df_pair["Match"] = df_pair["Predicted_Class"] == df_pair["Class"]
accuracy = df_pair["Match"].mean()

# Display results
display(df_pair[["Base_ID", "Side", "Y_star", "Y_Refined", "overbite_mm", "Predicted_Class", "Class", "Match"]])
print(f"✅ Classification Accuracy (side-matched): {accuracy:.2%}")
print(f"🔎 Matched image pairs (correct side): {len(df_pair)}")

# Optional: Save to CSV
df_pair.to_csv(r"C:\STUDIE\4. Semester\Dataprojekt\Dataprojekt - Solo\Overbite_Overjet\Output\Overbite Detection\NyeBilleder\55K\overbite_classification_results.csv", index=False)
print("💾 Results saved to overbite_classification_results_sidematched.csv")


,Base_ID,Side,Y_star,Y_Refined,overbite_mm,Predicted_Class,Class,Match
0,013FHA7K,left,390.0,369,1.68,B,B,True
1,013FHA7K,right,413.0,369,3.52,D,D,True
2,013NUWYR,left,305.0,243,4.96,E,E,True
3,013NUWYR,right,248.0,235,1.04,B,A,False
4,013NXP1H,left,368.0,357,0.88,A,A,True
...,...,...,...,...,...,...,...,...
145,LA67YYVJ,right,502.0,436,5.28,E,E,True
146,LGADV15G,left,360.0,350,0.80,A,A,True
147,LGADV15G,right,355.0,353,0.16,A,A,True
148,MNIAB8K3,left,354.0,336,1.44,B,B,True


✅ Classification Accuracy (side-matched): 95.33%
🔎 Matched image pairs (correct side): 150
💾 Results saved to overbite_classification_results_sidematched.csv
